# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [16]:
# Do all imports and installs here
import findspark
findspark.init('C:\spark')
               
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DecimalType

spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

### Step 1: Scope the Project and Gather Data

#### Scope 
In this open-ended project we build a Data Warehouse for the I94 Immigration Data coming from the US National Tourism and Trade Office, and further enrich it with various other datasets such as US City Demographic Data, Airports data, Country and States date, and Visa information.

#### Describe and Gather Data 
* **I94 Immigration Data**: This data comes from https://travel.trade.gov/research/reports/i94/historical/2016.html. There's a sample file so you can take a look at the data in csv format before reading it all in.

* **U.S. City Demographic Data**: This data comes from OpenSoft. You can read more about it at https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export

* **Airports Data**: This is a simple table of airport codes and corresponding cities. It comes from https://datahub.io/core/airport-codes#data

* **Others**: Some other datasets such as Country & State codes, and Visa information have been consolidated in an SAS file included here in this project: data/I94_SAS_Labels_Descriptions.SAS

In [2]:
# Read in the data here
fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")

In [3]:
df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


In [1]:
	
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [4]:
#write to parquet
#df_spark.write.parquet("immigration_data")

In [8]:
#read the data
df_visits=spark.read.parquet("data/immigration_data")
df_demo = spark.read.csv("data/us-cities-demographics.csv", sep=";", header=True)
df_airports = spark.read.csv("data/airport-codes_csv.csv", header=True)

### Step 2: Explore and Assess the Data
#### Explore the Data 

In [5]:
df_visits.show(4)
df_visits.count()

+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|    cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|5748517.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     CA|20582.0|  40.0|    1.0|  1.0|20160430|     SYD| null|      G|      O|   null|      M| 1976.0|10292016|     F|  null|     QF|9.495387003E10|00011|      B1|
|5748518.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     NV|20591.0|  32.0|    1.0|  

3096313

In [6]:
df_visits.describe()

DataFrame[summary: string, cicid: string, i94yr: string, i94mon: string, i94cit: string, i94res: string, i94port: string, arrdate: string, i94mode: string, i94addr: string, depdate: string, i94bir: string, i94visa: string, count: string, dtadfile: string, visapost: string, occup: string, entdepa: string, entdepd: string, entdepu: string, matflag: string, biryear: string, dtaddto: string, gender: string, insnum: string, airline: string, admnum: string, fltno: string, visatype: string]

In [11]:
df_airports.show(4)
df_airports.count()

+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|  00A|     heliport|   Total Rf Heliport|          11|       NA|         US|     US-PA|    Bensalem|     00A|     null|       00A|-74.9336013793945...|
| 00AA|small_airport|Aero B Ranch Airport|        3435|       NA|         US|     US-KS|       Leoti|    00AA|     null|      00AA|-101.473911, 38.7...|
| 00AK|small_airport|        Lowell Field|         450|       NA|         US|     US-AK|Anchor Point|    00AK|     null|      00AK|-151.695999146, 5...|
| 00AL|small_airport|        Epps Airpark|         820|       NA|         US|     

55075

In [12]:
df_airports.describe()

DataFrame[summary: string, ident: string, type: string, name: string, elevation_ft: string, continent: string, iso_country: string, iso_region: string, municipality: string, gps_code: string, iata_code: string, local_code: string, coordinates: string]

In [13]:
df_demo.show(4)
df_demo.count()

+----------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|            City|        State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|Average Household Size|State Code|                Race|Count|
+----------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|   Silver Spring|     Maryland|      33.8|          40601|            41862|           82463|              1562|       30908|                   2.6|        MD|  Hispanic or Latino|25924|
|          Quincy|Massachusetts|      41.0|          44129|            49500|           93629|              4147|       32935|                  2.39|        MA|               White|58723|
|          Hoover|      Alabama|      38.5|          38040| 

2891

In [14]:
df_demo.describe()

DataFrame[summary: string, City: string, State: string, Median Age: string, Male Population: string, Female Population: string, Total Population: string, Number of Veterans: string, Foreign-born: string, Average Household Size: string, State Code: string, Race: string, Count: string]

In [23]:
# parse SAS_labels_descriptions file for valid airport codes and read into a DataFrame
f = open("data/I94_SAS_Labels_Descriptions.SAS", "r")
lines = f.readlines()
airports=[]
for line in lines[302:893]:
    line_split = line.replace("'", "").split('=')
    airports.append({"airport_code":line_split[0].strip(), "airport_name":line_split[1].strip()})

schema = StructType([
    StructField('airport_code', StringType()),
    StructField('airport_name', StringType())
])

print(f"First airport: {airports[0]}, Last airport: {airports[-1]}")

df_airport_codes = spark.createDataFrame(airports, schema)
df_airport_codes.show(4)
df_airport_codes.count()

First airport: {'airport_code': 'ALC', 'airport_name': 'ALCAN, AK'}, Last airport: {'airport_code': 'YUM', 'airport_name': 'YUMA, AZ'}
+------------+--------------------+
|airport_code|        airport_name|
+------------+--------------------+
|         ALC|           ALCAN, AK|
|         ANC|       ANCHORAGE, AK|
|         BAR|BAKER AAF - BAKER...|
|         DAC|   DALTONS CACHE, AK|
+------------+--------------------+
only showing top 4 rows



591

In [24]:
# parse SAS_labels_descriptions file for valid country codes and read into a DataFrame
f = open("data/I94_SAS_Labels_Descriptions.SAS","r")
lines = f.readlines()
countries=[]
for line in lines[9:245]:
    line_split = line.replace("'", "").split('=')
    countries.append({"country_code":int(line_split[0].strip()), "country_name":line_split[1].strip()})

schema = StructType([
    StructField('country_code', IntegerType()),
    StructField('country_name', StringType())
])

print(f"First country: {countries[0]}, Last country: {countries[-1]}")

df_countries = spark.createDataFrame(countries, schema)
df_countries.show(4)
df_countries.count()

First country: {'country_code': 582, 'country_name': 'MEXICO Air Sea, and Not Reported (I-94, no land arrivals)'}, Last country: {'country_code': 315, 'country_name': 'ZIMBABWE'}
+------------+--------------------+
|country_code|        country_name|
+------------+--------------------+
|         582|MEXICO Air Sea, a...|
|         236|         AFGHANISTAN|
|         101|             ALBANIA|
|         316|             ALGERIA|
+------------+--------------------+
only showing top 4 rows



236

In [25]:
# parse SAS_labels_descriptions file for state codes and names and read into a DataFrame
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DecimalType
f = open("data/I94_SAS_Labels_Descriptions.SAS","r")
lines = f.readlines()
states=[]
for line in lines[981:1035]:
    line_split = line.replace("'", "").split('=')
    states.append({"code":line_split[0].strip(), "name":line_split[1].strip()})

schema = StructType([
    StructField('code', StringType()),
    StructField('name', StringType())
])

print(f"First state: {states[0]}, Last state: {states[-1]}")

df_states = spark.createDataFrame(states, schema)
df_states.show(4)
df_states.count()

First state: {'code': 'AL', 'name': 'ALABAMA'}, Last state: {'code': 'WY', 'name': 'WYOMING'}
+----+--------+
|code|    name|
+----+--------+
|  AL| ALABAMA|
|  AK|  ALASKA|
|  AZ| ARIZONA|
|  AR|ARKANSAS|
+----+--------+
only showing top 4 rows



54

In [26]:
# parse SAS_labels_descriptions file for visa types and read into a DataFrame
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DecimalType
f = open("data/I94_SAS_Labels_Descriptions.SAS","r")
lines = f.readlines()
visa=[]
for line in lines[1046:1049]:
    line_split = line.replace("'", "").split('=')
    visa.append({"id":int(line_split[0].strip()), "visa":line_split[1].strip()})

schema = StructType([
    StructField('id', IntegerType()),
    StructField('visa', StringType())
])

print(f"First visa: {visa[0]}, Last visa: {visa[-1]}")

df_visa = spark.createDataFrame(visa, schema)
df_visa.show(4)
df_visa.count()

First visa: {'id': 1, 'visa': 'Business'}, Last visa: {'id': 3, 'visa': 'Student'}
+---+--------+
| id|    visa|
+---+--------+
|  1|Business|
|  2|Pleasure|
|  3| Student|
+---+--------+



3

### Cleaning Steps
**Clean Immigration Data**
* Pick only relevant columns, rename columns to meaningful names
* Fix data types, date formats
* Discard invalid airport/country/state/visa codes

In [28]:
df_visits.createOrReplaceTempView("visits")
df_airport_codes.createOrReplaceTempView("airport_codes")
df_countries.createOrReplaceTempView("countries")
df_states.createOrReplaceTempView("states")
df_visa.createOrReplaceTempView("visa")

df_visits_format = spark.sql("""SELECT cast(cicid as int) id, cast(i94cit as int) country_citizenship, 
cast(i94res as int) country_residence, i94port port, date_add('1960-01-01', cast(arrdate as int)) arrival_date, 
date_add('1960-01-01', cast(depdate as int)) departure_date, I94addr state, cast(i94bir as int) age,gender, 
cast(i94visa as int) visa, airline FROM visits 
""")

df_visits_format.createOrReplaceTempView("visits")

df_visits_clean = spark.sql("""SELECT id, country_citizenship, country_residence, port, arrival_date, 
departure_date, state, age,gender, visa,airline FROM visits 
where port in (select airport_code from airport_codes)
and country_residence in (select country_code from countries)
and country_citizenship in (select country_code from countries)
and state in (select code from states)
and visa in (select id from visa)
""")

#df_visits_clean.createOrReplaceTempView("visits")
df_visits_clean.show(4)
df_visits_clean.count()

+-------+-------------------+-----------------+----+------------+--------------+-----+---+------+----+-------+
|     id|country_citizenship|country_residence|port|arrival_date|departure_date|state|age|gender|visa|airline|
+-------+-------------------+-----------------+----+------------+--------------+-----+---+------+----+-------+
|2959311|                516|              516| FTL|  2016-04-16|    2016-04-22|   AZ| 41|     M|   1|     BW|
|4563478|                516|              516| HOU|  2016-04-24|    2016-04-27|   AZ| 32|     M|   1|     UA|
|4563479|                516|              516| HOU|  2016-04-24|    2016-05-01|   AZ| 38|     M|   1|     UA|
|1849132|                251|              251| DET|  2016-04-10|    2016-04-15|   AZ| 40|     M|   1|     UA|
+-------+-------------------+-----------------+----+------------+--------------+-----+---+------+----+-------+
only showing top 4 rows



2550725

**Clean Airport Data**
* Pick only relevant columns
* Discard non-US airports and rows with missing local_code

In [29]:
df_airports.createOrReplaceTempView("airports")

df_airports_clean = spark.sql("""
select name, iso_country, iso_region, municipality, iata_code, local_code
from airports where iso_country='US' and local_code is not null
""")

#df_airports_clean.createOrReplaceTempView("airports")
df_airports_clean.show(4)
df_airports_clean.count()

+--------------------+-----------+----------+------------+---------+----------+
|                name|iso_country|iso_region|municipality|iata_code|local_code|
+--------------------+-----------+----------+------------+---------+----------+
|   Total Rf Heliport|         US|     US-PA|    Bensalem|     null|       00A|
|Aero B Ranch Airport|         US|     US-KS|       Leoti|     null|      00AA|
|        Lowell Field|         US|     US-AK|Anchor Point|     null|      00AK|
|        Epps Airpark|         US|     US-AL|     Harvest|     null|      00AL|
+--------------------+-----------+----------+------------+---------+----------+
only showing top 4 rows



21236

**Clean Demographics Data**
* Pick only relevant columns, rename columns, remove spaces in column names, fix column data types
* Discard rows with missing city or state value
* Remove duplicates (by City and State) 

In [31]:
df_demo.createOrReplaceTempView("demo")

df_demo_clean = spark.sql("""select distinct city,`State Code` state,cast(`Median Age` as float) median_age, 
cast(`Male Population` as int) male_population,cast(`Female Population` as int) female_population, 
cast(`Total Population` as int) total_population, cast(`Foreign-born` as int) foreign_born from demo 
where state is not null and city is not null
""")

df_demo_clean.show(4)
df_demo_clean.count()

#df_demo_format.createOrReplaceTempView("demo")

+-----------+-----+----------+---------------+-----------------+----------------+------------+
|       city|state|median_age|male_population|female_population|total_population|foreign_born|
+-----------+-----+----------+---------------+-----------------+----------------+------------+
|  Harlingen|   TX|      30.1|          32404|            33365|           65769|       10391|
|Brownsville|   TX|      30.6|          87689|            96199|          183888|       53301|
|    Sunrise|   FL|      37.7|          41471|            51235|           92706|       38209|
|  Vacaville|   CA|      35.8|          50091|            46703|           96794|       10577|
+-----------+-----+----------+---------------+-----------------+----------------+------------+
only showing top 4 rows



596

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model



### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.